## CSCI 303
# Introduction to Data Science
<p/>
### 13 - Working with SQL Databases (2)

![Relational database icon](sql.png)

## This Lecture
---
- More SELECT queries
  - Operators and functions
  - Sorting
  - Grouping and aggregating
  - Joining tables

The obligatory setup code...

In [1]:
import pandas as pd
import sqlite3   # We'll be using a simple file-based SQLite3 database
from pandas import Series, DataFrame

dburi = 'sqlite:///csci303.sqlite3'

Let's also display info about the tables we'll be using in our examples:

In [2]:
from sqlalchemy import create_engine, inspect
inspector = inspect(create_engine(dburi))

In [3]:
pd.DataFrame(inspector.get_columns('scifi_author'))

,name,type,nullable,default,autoincrement,primary_key
0,id,TEXT,False,None,auto,1
1,name,TEXT,False,None,auto,0
2,birth_year,INTEGER,True,None,auto,0
3,death_year,INTEGER,True,None,auto,0


In [4]:
pd.DataFrame(inspector.get_columns('scifi_work'), columns=['name','type'])

,name,type
0,id,TEXT
1,author_id,TEXT
2,title,TEXT
3,publication_year,INTEGER


## Operators and Functions
---
Wildcard matching using `LIKE`:

In [5]:
pd.read_sql_query("SELECT * FROM scifi_author WHERE name LIKE 'A%%'", dburi) # searches for all authors that start with an A

,id,name,birth_year,death_year
0,/authors/OL3399168A,Ann Leckie,NaN,NaN
1,/authors/OL24708A,A. E. van Vogt,1912.0,2000.0
2,/authors/OL7304221A,Arthur C. Clarke,NaN,NaN
3,/authors/OL2623461A,Arthur C. Clarke,1917.0,2008.0
4,/authors/OL7228383A,Ann Leckie,NaN,NaN
5,/authors/OL7099704A,Andre Norton,1912.0,2005.0
6,/authors/OL7319169A,Arthur C. Clarke,NaN,NaN
7,/authors/OL218124A,Alfred Bester,1913.0,1987.0


The '%' wildcard matches any string of any length, so the above query asks for all authors where the author name starts with an 'A'.

Note that the wildcard in SQL is just a single '%', but that has meaning in Python strings, so we have to escape it by using an extra '%'.

You can also use '_' to stand in for any single character.

So, for instance, we could get authors for whom we have only a first initial:

In [6]:
pd.read_sql_query("SELECT * FROM scifi_author WHERE name LIKE '_. %%'", dburi)

,id,name,birth_year,death_year
0,/authors/OL7046811A,C. J. Cherryh,NaN,NaN
1,/authors/OL24708A,A. E. van Vogt,1912.0,2000.0
2,/authors/OL7313078A,C. J. Cherryh,NaN,NaN
3,/authors/OL7342190A,C. J. Cherryh,NaN,NaN
4,/authors/OL7319469A,C. J. Cherryh,NaN,NaN


SQL has numerous functions and operators.  For instance, pretty much any mathematical expression is allowed:

In [7]:
# gets the name of the author
# also gets the age of the author by subtracting birth year from death yaer (as long as that person is alive)
query = """
 SELECT name, death_year - birth_year AS approx_age 
 FROM scifi_author 
 WHERE death_year IS NOT NULL
"""

pd.read_sql_query(query, dburi)

,name,approx_age
0,A. E. van Vogt,88
1,Arthur C. Clarke,91
2,Cordwainer Smith,53
3,Robert A. Heinlein,81
4,Clifford D. Simak,84
5,Roger Zelazny,58
6,Isaac Asimov,72
7,Andre Norton,93
8,Alfred Bester,74
9,Philip K. Dick,54


We slipped in a couple of other SQL things there.  Let's explain those.

First, the `AS` keyword lets us rename or "alias" a column.  That's handy when doing an expression like above.

We'll see other uses for `AS` later.

Also, we used another operator, `IS NOT NULL`.

NULL values in SQL are not comparable - any attempt to compare them using a relational operator (such as =) will always return false!

Instead, use `IS NULL` and `IS NOT NULL` to accept/reject NULL values.

There are a number of useful string functions and operators.

For example, is it "A. E. Van Vogt", or "A. E. van Vogt"?

We can convert strings to all lowercase using `lower`:

In [8]:
pd.read_sql_query("SELECT * FROM scifi_author WHERE lower(name) LIKE '%%van vogt'", dburi)
#pd.read_sql_query("SELECT * FROM scifi_author WHERE name LIKE '%%van vogt'", dburi)

,id,name,birth_year,death_year
0,/authors/OL24708A,A. E. van Vogt,1912,2000


All of the functions and operators mentioned above are standard SQL.

Your database may supply (many) others in addition.

For example, here's a link to documentation on PostgreSQL's functions and operators:
https://www.postgresql.org/docs/9.5/static/functions.html

## Sorting
---
To sort the results of a query, add an `ORDER BY` clause.

`ORDER BY` is followed by the columns you want to sort by.  Each column name can optionally be followed with `ASC` (the default) or `DESC` to determine whether the sort is ascending or descending.

In [9]:
pd.read_sql_query("SELECT * FROM scifi_author ORDER BY name", dburi)[:10] # sorts the database by name

,id,name,birth_year,death_year
0,/authors/OL24708A,A. E. van Vogt,1912.0,2000.0
1,/authors/OL218124A,Alfred Bester,1913.0,1987.0
2,/authors/OL7099704A,Andre Norton,1912.0,2005.0
3,/authors/OL3399168A,Ann Leckie,NaN,NaN
4,/authors/OL7228383A,Ann Leckie,NaN,NaN
5,/authors/OL7304221A,Arthur C. Clarke,NaN,NaN
6,/authors/OL2623461A,Arthur C. Clarke,1917.0,2008.0
7,/authors/OL7319169A,Arthur C. Clarke,NaN,NaN
8,/authors/OL7046811A,C. J. Cherryh,NaN,NaN
9,/authors/OL7313078A,C. J. Cherryh,NaN,NaN


In [10]:
# sorts the age of the author in descending order
query = """
 SELECT name, death_year - birth_year AS approx_age 
 FROM scifi_author 
 WHERE death_year IS NOT NULL
 ORDER BY approx_age DESC 
"""

pd.read_sql_query(query, dburi) 

,name,approx_age
0,Andre Norton,93
1,Arthur C. Clarke,91
2,A. E. van Vogt,88
3,Clifford D. Simak,84
4,Robert A. Heinlein,81
5,Poul Anderson,75
6,Alfred Bester,74
7,Isaac Asimov,72
8,Roger Zelazny,58
9,Philip K. Dick,54


## DISTINCT
---
Somewhat related to sorting, sometimes you want to get only a unique set of records back.

This is particularly useful when trying to find the unique settings for a particular column.

For instance, our data unfortunately has a lot of duplicates.

Let's see just the unique author names starting with 'C':

In [11]:
query = """
SELECT DISTINCT name
FROM scifi_author
WHERE name LIKE 'C%%'
"""

pd.read_sql_query(query, dburi)

,name
0,Clifford D. Simak
1,C. J. Cherryh
2,Connie Willis
3,Cordwainer Smith
4,Charles Stross


## Grouping and Aggregating
---
SQL has capabilities for grouping data and computing aggregate functions on the groups, very similar to what we saw pandas doing in the Advanced pandas lecture.

The basic syntax is

```
SELECT a1, a2, ..., fn1(a3), fn2(a4), ...
FROM table
GROUP BY a1, a2, ...;
```

where `fn1` etc. compute some kind of aggregate.  Some of the functions available are `COUNT`, `SUM`, `AVG`, `MIN`, and `MAX`.

It is important that every attribute selected participate in the group operation.

For example, this query is valid and finds out the multiplicity of each author in our database:

In [12]:
query = """
SELECT name, COUNT(name) FROM scifi_author
GROUP BY name
"""

pd.read_sql_query(query, dburi)

,name,COUNT(name)
0,A. E. van Vogt,1
1,Alfred Bester,1
2,Andre Norton,1
3,Ann Leckie,2
4,Arthur C. Clarke,3
5,C. J. Cherryh,4
6,Charles Stross,1
7,Clifford D. Simak,3
8,Connie Willis,1
9,Cordwainer Smith,2


whereas this query is invalid, because we are not grouping by birth_year:

In [13]:
query = """
SELECT name, COUNT(name), birth_year FROM scifi_author
GROUP BY name
"""
pd.read_sql_query(query, dburi)

,name,COUNT(name),birth_year
0,A. E. van Vogt,1,1912.0
1,Alfred Bester,1,1913.0
2,Andre Norton,1,1912.0
3,Ann Leckie,2,NaN
4,Arthur C. Clarke,3,NaN
5,C. J. Cherryh,4,NaN
6,Charles Stross,1,1964.0
7,Clifford D. Simak,3,NaN
8,Connie Willis,1,NaN
9,Cordwainer Smith,2,1913.0


Usefully, we can also `ORDER BY` aggregate expressions. Suppose we want the most duplicated authors (to see where we need to do the most cleanup):

In [14]:
query = """
SELECT name, COUNT(name) FROM scifi_author
GROUP BY name
ORDER BY COUNT(name) DESC
"""
pd.read_sql_query(query, dburi)

,name,COUNT(name)
0,C. J. Cherryh,4
1,Clifford D. Simak,3
2,Arthur C. Clarke,3
3,Roger Zelazny,2
4,Robert Silverberg,2
5,Philip K. Dick,2
6,Julian May,2
7,Isaac Asimov,2
8,Frederik Pohl,2
9,Cordwainer Smith,2


We can also use another clause with the keyword `HAVING`, which lets us filter the grouped results according to aggregate values.  Let's only see the authors with multiplicity greater than 2:

In [15]:
query = """
SELECT name, COUNT(name) FROM scifi_author
GROUP BY name
HAVING COUNT(name) > 2
"""
pd.read_sql_query(query, dburi)

,name,COUNT(name)
0,Arthur C. Clarke,3
1,C. J. Cherryh,4
2,Clifford D. Simak,3


Aggregate functions can also be applied without grouping:

In [16]:
query = """
 SELECT 
   MIN(death_year - birth_year),
   AVG(death_year - birth_year),
   MAX(death_year - birth_year) 
 FROM scifi_author 
 WHERE death_year IS NOT NULL
"""

pd.read_sql_query(query, dburi)

,MIN(death_year - birth_year),AVG(death_year - birth_year),MAX(death_year - birth_year)
0,53,74.818182,93


## Joins
---
Databases are often factored in such a way as to minimize duplicate information.  (This database didn't succeed so well in that.)

For example, we have a table of Science Fiction works, but it doesn't include author names or dates.  

Rather, the authors live in a separate table, and are simply referenced by a key field from the works table.

In [17]:
pd.DataFrame(inspector.get_columns('scifi_work'), columns=['name','type'])

,name,type
0,id,TEXT
1,author_id,TEXT
2,title,TEXT
3,publication_year,INTEGER


The `author_id` field provides our linkage to the `scifi_author` table.

There are two ways to do inner joins, which are the joins we most commonly want to do.

Here's the "wordy" way to join `scifi_author` and `scifi_work`:

In [18]:
query = """
SELECT scifi_author.name, scifi_work.title, scifi_work.publication_year
FROM   scifi_author JOIN scifi_work ON scifi_author.id = scifi_work.author_id
"""

pd.read_sql_query(query, dburi)

,name,title,publication_year
0,Poul Anderson,Genesis,NaN
1,Julian May,Metaconcert,1989.0
2,Roger Zelazny,Sign of Chaos,1987.0
3,Arthur C. Clarke,A Meeting With Medusa/Green Mars (Special Doub...,NaN
4,Arthur C. Clarke,The Space Trilogy,NaN
...,...,...,...
4930,Robert Silverberg,"Le Seigneur des ténèbres, tome 2",1996.0
4931,Robert Silverberg,Star of Gypsies,1986.0
4932,Philip K. Dick,Time Out of Joint,2003.0
4933,A. E. van Vogt,The Silkie,NaN


Some of the table name specifiers are unnecessary in the above query, as SQL can work out what table you mean if the column name is unique.

The only column we really *need* the specifier on is the `scifi_author.id` column, since there is also a column named `id` in `scifi_work`.

However, it makes things clearer if we do something to specify what tables columns are coming from.

This is a common use of table aliases using the `AS` keyword.

In [19]:
query = """
SELECT a.name, w.title, w.publication_year
FROM   scifi_author AS a JOIN scifi_work AS w
ON a.id = w.author_id
"""

pd.read_sql_query(query, dburi)

,name,title,publication_year
0,Poul Anderson,Genesis,NaN
1,Julian May,Metaconcert,1989.0
2,Roger Zelazny,Sign of Chaos,1987.0
3,Arthur C. Clarke,A Meeting With Medusa/Green Mars (Special Doub...,NaN
4,Arthur C. Clarke,The Space Trilogy,NaN
...,...,...,...
4930,Robert Silverberg,"Le Seigneur des ténèbres, tome 2",1996.0
4931,Robert Silverberg,Star of Gypsies,1986.0
4932,Philip K. Dick,Time Out of Joint,2003.0
4933,A. E. van Vogt,The Silkie,NaN


An equivalent, and more compact way to write the same query is to move the join condition(s) into the WHERE clause, and simply list the tables you want data from:



In [20]:
query = """
SELECT a.name, w.title, w.publication_year
FROM scifi_author AS a, scifi_work AS w
WHERE a.id = w.author_id
"""
pd.read_sql_query(query, dburi)

,name,title,publication_year
0,Poul Anderson,Genesis,NaN
1,Julian May,Metaconcert,1989.0
2,Roger Zelazny,Sign of Chaos,1987.0
3,Arthur C. Clarke,A Meeting With Medusa/Green Mars (Special Doub...,NaN
4,Arthur C. Clarke,The Space Trilogy,NaN
...,...,...,...
4930,Robert Silverberg,"Le Seigneur des ténèbres, tome 2",1996.0
4931,Robert Silverberg,Star of Gypsies,1986.0
4932,Philip K. Dick,Time Out of Joint,2003.0
4933,A. E. van Vogt,The Silkie,NaN


Now we can ask questions like, "What books did Ann Leckie write?"

In [21]:
query = """
SELECT a.name, w.title, w.publication_year
FROM scifi_author AS a, scifi_work AS w
WHERE a.id = w.author_id
AND a.name = 'Ann Leckie'
"""
pd.read_sql_query(query, dburi)

,name,title,publication_year
0,Ann Leckie,Ancillary Justice,None
1,Ann Leckie,Ancillary Mercy,None
2,Ann Leckie,Ancillary Sword,None


Or, "Who wrote *I, Robot*?"

In [22]:
query = """
SELECT a.name
FROM scifi_author AS a, scifi_work AS w
WHERE a.id = w.author_id
AND w.title = 'I, Robot'
"""
pd.read_sql_query(query, dburi)

,name
0,Isaac Asimov
1,Isaac Asimov


## Putting It All Together
---
Now that we know the basics, let's try some queries with the Sci-fi books dataset.

Let's start with, "How many books are listed for each author entry?"

In [23]:
query = """
SELECT a.id, a.name, COUNT(w.title)
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
GROUP BY a.id, a.name
ORDER BY a.name
"""
pd.read_sql_query(query, dburi)

,id,name,COUNT(w.title)
0,/authors/OL24708A,A. E. van Vogt,208
1,/authors/OL218124A,Alfred Bester,34
2,/authors/OL7099704A,Andre Norton,285
3,/authors/OL7228383A,Ann Leckie,3
4,/authors/OL2623461A,Arthur C. Clarke,376
5,/authors/OL7304221A,Arthur C. Clarke,1
6,/authors/OL7319169A,Arthur C. Clarke,2
7,/authors/OL7046811A,C. J. Cherryh,165
8,/authors/OL7313078A,C. J. Cherryh,1
9,/authors/OL7319469A,C. J. Cherryh,1


That's probably not what we wanted; our data has some bad duplication in it.

Let's try just grouping by author name:


In [24]:
query = """
SELECT a.name, COUNT(w.title)
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
GROUP BY a.name
ORDER BY a.name
"""
pd.read_sql_query(query, dburi)

,name,COUNT(w.title)
0,A. E. van Vogt,208
1,Alfred Bester,34
2,Andre Norton,285
3,Ann Leckie,3
4,Arthur C. Clarke,379
5,C. J. Cherryh,168
6,Charles Stross,38
7,Clifford D. Simak,138
8,Connie Willis,51
9,Cordwainer Smith,27


I wonder if duplication is a problem in the works data, too?

Let's look closer at one of our more prolific authors:

In [25]:
query = """
SELECT w.title, COUNT(w.title)
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
AND    a.name = 'Arthur C. Clarke'
GROUP BY w.title
HAVING COUNT(w.title) > 1
ORDER BY w.title
"""
pd.read_sql_query(query, dburi)

,title,COUNT(w.title)
0,1984,2
1,2001,6
2,"2001, a space odyssey",2
3,2010,3
4,"2061, [i.e. Dos mil sesenta y uno] odisea tres",5
5,2061: Odisei III,5
6,Across the Sea of Stars,2
7,American heritage,2
8,Beyond the fall of night,2
9,Cánticos de la lejanatierra,5


Hm, maybe we can filter this down a bit.  It turns out you can combine COUNT and DISTINCT - this will at least let us remove books with exact duplicate titles.

(It still won't help with books that are the same but are listed differently, or books in other languages.)

In [26]:
query = """
SELECT a.name, COUNT(w.title), COUNT(DISTINCT w.title)
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
GROUP BY a.name
ORDER BY a.name
"""
pd.read_sql_query(query, dburi)

,name,COUNT(w.title),COUNT(DISTINCT w.title)
0,A. E. van Vogt,208,153
1,Alfred Bester,34,31
2,Andre Norton,285,275
3,Ann Leckie,3,3
4,Arthur C. Clarke,379,291
5,C. J. Cherryh,168,163
6,Charles Stross,38,32
7,Clifford D. Simak,138,111
8,Connie Willis,51,44
9,Cordwainer Smith,27,24


If we want to see the actual distinct titles, we can just use DISTINCT without grouping:

In [27]:
query = """
SELECT DISTINCT a.name, w.title
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
ORDER BY a.name, w.title
"""
pd.read_sql_query(query, dburi)

,name,title
0,A. E. van Vogt,'O Kosmos tou Meden-A
1,A. E. van Vogt,A Report on the Violent Male
2,A. E. van Vogt,A Van Vogt omnibus
3,A. E. van Vogt,A la poursuite des Slans
4,A. E. van Vogt,Anagennese
...,...,...
4083,Roger Zelazny,Wilderness
4084,Roger Zelazny,Wizard World
4085,Roger Zelazny,Wizard world
4086,Roger Zelazny,l'île des morts


"Which of our authors is most prolific?"

In [28]:
query = """
SELECT a.name, COUNT(DISTINCT w.title) AS unique_count
FROM   scifi_author AS a, scifi_work AS w
WHERE  a.id = w.author_id
GROUP BY a.name
ORDER BY unique_count DESC
"""
pd.read_sql_query(query, dburi)[:5]

,name,unique_count
0,Isaac Asimov,1226
1,Robert Silverberg,475
2,Arthur C. Clarke,291
3,Andre Norton,275
4,Poul Anderson,254


"What year had the most works published?"

In [29]:
query = """
SELECT publication_year, COUNT(DISTINCT title) AS unique_count
FROM scifi_work
GROUP BY publication_year
ORDER BY unique_count DESC
"""
pd.read_sql_query(query, dburi)[:5]

,publication_year,unique_count
0,NaN,3267
1,1972.0,47
2,1975.0,37
3,1973.0,36
4,1999.0,33


"What years did Arthur C. Clarke publish the most in?"

In [30]:
query = """
SELECT w.publication_year, COUNT(DISTINCT w.title) AS unique_count
FROM scifi_work AS w, scifi_author AS a
WHERE a.id = w.author_id AND a.name = 'Arthur C. Clarke'
GROUP BY publication_year
ORDER BY unique_count DESC
"""
pd.read_sql_query(query, dburi)[:5]

,publication_year,unique_count
0,NaN,241
1,1972.0,5
2,1957.0,5
3,1989.0,4
4,1953.0,4


We clearly don't know more than we know...

What other questions can you think to ask of this data?

## Extra Stuff 
---
- SQL odds and ends
  - Set operations
  - Subqueries
  - Outer joins